In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import torch
import soundfile
from utils import torch_device_seed, check_gpu, clear_device_cache

from tqdm.notebook import tqdm

In [ ]:
def check_gpu():
    # torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
    is_cuda = torch.cuda.is_available()
    
    is_mps = torch.backends.mps.is_available()

    # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
    if is_cuda:
        device = torch.device("cuda")
        print("GPU is available")
    else:
        if is_mps:
            device = torch.device("mps")
        else:
            device = torch.device("cpu")
            print("GPU not available, CPU used")
    return device

In [ ]:
device = check_gpu()
torch_device_seed(3407)
print(device)

In [ ]:
def get_waveforms(file):
    wave,_= librosa.load(file,duration=3,offset=0.5,sr=sample_rate)
    waveform, _ = librosa.effects.trim(wave,top_db=60) # tops at 60 decibels

    # make sure waveform vectors are homogenous by defining explicitly
    waveform_homo = np.zeros((int(sample_rate*3,)))
    waveform_homo[:len(waveform)] = waveform
    
    # return a single file's waveform    
    return waveform_homo

In [ ]:
emotions_dict = {
    0:'surprised',
    1:'neutral',
    2:'calm',
    3:'happy',
    4:'sad',
    5:'angry',
    6:'fearful',
    7:'disgust'
}
# Additional attributes from RAVDESS to play with
emotion_intensities = {
    1: 'normal',
    2: 'strong'
}

# RAVDESS native sample rate is 48k
sample_rate = 48000

In [ ]:
# Function to extract features
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        stft = np.abs(librosa.stft(X)) if chroma else None
        result=np.array([])
        
        if mfcc:
            mfcc_vector = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
            if len(mfcc_vector.shape) > 2:
                mfcc_vector = mfcc_vector.squeeze(2)
                mfcc_vector = mfcc_vector.reshape(40, -1) # n_mfcc parameter
            mfccs = np.mean(mfcc_vector.T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma_vector = librosa.feature.chroma_stft(S=stft, sr=sample_rate)
            if len(chroma_vector.shape) > 2:
                chroma_vector = chroma_vector.squeeze(2)
                chroma_vector = chroma_vector.reshape(12, -1) # n_chroma_bins parameter
            chroma_feature = np.mean(chroma_vector.T, axis=0)
            result=np.hstack((result, chroma_feature))
        if mel:
            mel_vector = librosa.feature.melspectrogram(y=X, sr=sample_rate)
            if len(mel_vector.shape) > 2:
                mel_vector = mel_vector.squeeze(2)
                mel_vector = mel_vector.reshape(128, -1) # n_mels parameter
            mel_feature = np.mean(mel_vector.T, axis=0)
            result=np.hstack((result, mel_feature))
        print(mfccs.shape, chroma_feature.shape, mel_feature.shape)
    return result

In [ ]:
RAV = "ravdess-emotional-speech-audio/"

dir_list = os.listdir(RAV)
dir_list.sort()
# features and labels
emotions = []
features = []
# raw waveforms to augment later
# waveforms = []
# extra labels
intensities, genders = [],[]

for i in tqdm(dir_list, total=len(dir_list)):
    if i == ".DS_Store":
        continue

    if not i.startswith("Actor_"):
        continue

    fname = os.listdir(RAV+i)
    for f in fname:
        # Modality (01 = full-AV, 02 = video-only, 03 = audio-only). We only have 03
        # Vocal channel (01 = speech, 02 = song). We only have 01
        # Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
        # Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
        # Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
        # Repetition (01 = 1st repetition, 02 = 2nd repetition).
        # Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).
        part = f.split('.')[0].split('-')
        
        emotion = int(part[2])
        
        file_dir = RAV + i +'/'+ f
        print('file:',file_dir)
        
        #  move surprise to 0 for cleaner behaviour with PyTorch/0-indexing
        if emotion == 8: 
            emotion = 0 # surprise is now at 0 index; other emotion indeces unchanged
        elif emotion == 2: 
            continue # Skip calm emotion
        
        intensity = int(part[3])
        
        gender = int(part[6])
        if gender%2==0:
            gender = 'female'
        else:
            gender = 'male'
        
        # get waveform from the sample
        # waveform = get_waveforms(file_dir)
        
        feature = extract_feature(file_dir, mfcc = True, chroma = True, mel = True)
        # store waveforms and labels
        features.append(feature)
        emotions.append(emotion) # no need for the onehot encoded
        intensities.append(intensity) # store intensity in case we wish to predict
        genders.append(gender)

In [ ]:
emotions[0]

In [ ]:
intensities[0]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid_test, y_train, y_valid_test = train_test_split(features, emotions, test_size=0.2, stratify=emotions)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class WaveformDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [ ]:
train_ds = WaveformDataset(X_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=32, num_workers=0, shuffle=True)

In [ ]:
for x in train_dataloader:
    print(x)
    break

In [ ]:
valid_ds = WaveformDataset(X_valid_test, y_valid_test)
valid_dataloader = DataLoader(valid_ds, batch_size=32, num_workers=0, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class SimpleLinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleLinearModel, self).__init__()

        self.lstm = nn.LSTM(input_size, 128, num_layers=1, dropout=0.1, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(128 * 2, 256) # for bidirectional
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, output_size)

    
    def forward(self, waveform):
        output, _ = self.lstm(waveform)
        output = F.relu(self.linear1(output))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)

        return output

In [ ]:
import gc

clear_device_cache()

gc.collect()

In [ ]:
linear_model = SimpleLinearModel(features[0].shape[0], len(emotions_dict)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_model.parameters(), lr=0.001)

In [ ]:
EPOCHS = 20

In [ ]:
def fit(model, criterion, optimizer, epochs=EPOCHS):
    train_accuracies = []
    val_accuracies = []
    train_losses = []
    val_losses = []

    for epoch in range(epochs):  # Loop over the dataset multiple times
        running_loss = 0.0
        total = 0
        correct = 0
        model.train()
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device).float()
            labels = labels.to(device)
    
            optimizer.zero_grad()
    
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item() 

            _, predicted = torch.max(outputs.data, axis=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        # get train loss and accuracy
        train_loss = running_loss / len(train_dataloader.dataset)
        train_accuracy = correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
    
        # get test loss and accuracy
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            model.eval()
            for data in valid_dataloader:
                inputs, labels = data
                inputs = inputs.to(device).float()
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, axis=1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(valid_dataloader)
        val_accuracy = val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch}: train_loss: {train_loss:.4f}; train_accuracy: {train_accuracy:.4f}; val_loss: {val_loss:.4f}; val_accuracy: {val_accuracy:.4f}")

    return {
        "train_loss": train_losses,
        "train_accuracy": train_accuracies,
        "val_loss": val_losses,
        "val_accuracy": val_accuracies
    }

In [ ]:
train_result = fit(linear_model, criterion, optimizer)

In [ ]:
from IPython.display import Audio

In [ ]:
sr = 22050

y_sweep = librosa.chirp(fmin=librosa.note_to_hz('C#4'),
                        fmax=librosa.note_to_hz('Eb5'),
                        sr=sr,
                        duration=1)

display(Audio(data=y_sweep, rate=sr))

In [ ]:
for x in train_dataloader:
    print(x[0].shape)
    break

In [ ]:
def predict(model, filename):
    f = filename.split("/")[-1]
    part = f.split('.')[0].split('-')
    emotion = int(part[2])

    emotion_label = emotions_dict[emotion]
    this_audio, sr = librosa.load(filename)

    feature = extract_feature(filename, mfcc = True, chroma = True, mel = True)

    with torch.no_grad():
        model.eval()
        feature_tensor = torch.tensor(np.array([feature]), dtype=torch.float).to(device)
        output = model(feature_tensor)
        _, prediction = torch.max(output, axis=1)
        prediction_label = emotions_dict[prediction.cpu().numpy()[0]]

    # show
    display(Audio(data=this_audio, rate=sr))
    print(f"Label     : {emotion_label}")
    print(f"Prediction: {prediction_label}")

In [ ]:
predict(linear_model, "ravdess-emotional-speech-audio/Actor_24/03-01-07-01-01-02-24.wav")